In [1]:
import pandas as pd
import requests as r
import sqlalchemy
import time
import datetime as dt
pd.options.display.max_columns = None

In [2]:
user_agent={"User-agent":"super-data-getter"}
after=""
data=[]
url="https://www.reddit.com/r/wallstreetbets/top/.json?t=day&after="

In [3]:
res = r.get(url, headers=user_agent).json()

In [4]:
while res['data'].get('after'):
    after = res['data'].get('after')
    res = r.get(f"{url}{after}", headers=user_agent).json()
    print(f"{url}{after}")
    for d in res['data']['children']:
        data.append(d)
    time.sleep(0.1)
len(data)

https://www.reddit.com/r/wallstreetbets/top/.json?t=day&after=t3_mrgyq0
https://www.reddit.com/r/wallstreetbets/top/.json?t=day&after=t3_mrn99m
https://www.reddit.com/r/wallstreetbets/top/.json?t=day&after=t3_mrmzzh


62

In [5]:
data[0]['data']

{'approved_at_utc': None,
 'subreddit': 'wallstreetbets',
 'selftext': '',
 'author_fullname': 't2_98d64knh',
 'saved': False,
 'mod_reason_title': None,
 'gilded': 0,
 'clicked': False,
 'title': 'Dave Portnoy-backed Buzz ETF adds GameStop, Palantir, and Chewy in monthly rebalancing',
 'link_flair_richtext': [{'e': 'text', 't': 'News'}],
 'subreddit_name_prefixed': 'r/wallstreetbets',
 'hidden': False,
 'pwls': 7,
 'link_flair_css_class': 'news',
 'downs': 0,
 'thumbnail_height': 70,
 'top_awarded_type': None,
 'hide_score': False,
 'name': 't3_mrm41b',
 'quarantine': False,
 'link_flair_text_color': 'light',
 'upvote_ratio': 0.81,
 'author_flair_background_color': None,
 'subreddit_type': 'public',
 'ups': 334,
 'total_awards_received': 0,
 'media_embed': {},
 'thumbnail_width': 140,
 'author_flair_template_id': None,
 'is_original_content': False,
 'user_reports': [],
 'secure_media': None,
 'is_reddit_media_domain': False,
 'is_meta': False,
 'category': None,
 'secure_media_embed'

In [6]:
df=pd.DataFrame([d['data'] for d in data ], dtype="string")

In [7]:
df.sample(5)

,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,link_flair_richtext,subreddit_name_prefixed,hidden,pwls,link_flair_css_class,downs,thumbnail_height,top_awarded_type,hide_score,name,quarantine,link_flair_text_color,upvote_ratio,author_flair_background_color,subreddit_type,ups,total_awards_received,media_embed,thumbnail_width,author_flair_template_id,is_original_content,user_reports,secure_media,is_reddit_media_domain,is_meta,category,secure_media_embed,link_flair_text,can_mod_post,score,approved_by,author_premium,thumbnail,edited,author_flair_css_class,author_flair_richtext,gildings,post_hint,content_categories,is_self,mod_note,created,link_flair_type,wls,removed_by_category,banned_by,author_flair_type,domain,allow_live_comments,selftext_html,likes,suggested_sort,banned_at_utc,url_overridden_by_dest,view_count,archived,no_follow,is_crosspostable,pinned,over_18,preview,all_awardings,awarders,media_only,link_flair_template_id,can_gild,spoiler,locked,author_flair_text,treatment_tags,visited,removed_by,num_reports,distinguished,subreddit_id,mod_reason_by,removal_reason,link_flair_background_color,id,is_robot_indexable,report_reasons,author,discussion_type,num_comments,send_replies,whitelist_status,contest_mode,mod_reports,author_patreon_flair,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,is_gallery,media_metadata,gallery_data,author_cakeday
61,<NA>,wallstreetbets,Your daily trading discussion thread. Please k...,t2_6l4z3,False,<NA>,0,False,"Unpinned Daily Discussion Thread for April 15,...","[{'e': 'text', 't': 'Discussion'}]",r/wallstreetbets,False,7,question,0,<NA>,<NA>,False,t3_mrbmiq,False,light,0.28,<NA>,public,0,1,{},<NA>,<NA>,False,[],<NA>,False,False,<NA>,{},Discussion,False,0,<NA>,True,self,False,<NA>,[],{'gid_1': 1},self,<NA>,True,<NA>,1618509615.0,richtext,7,<NA>,<NA>,text,self.wallstreetbets,True,"&lt;!-- SC_OFF --&gt;&lt;div class=""md""&gt;&lt...",<NA>,new,<NA>,<NA>,<NA>,False,True,False,False,False,{'images': [{'source': {'url': 'https://extern...,"[{'giver_coin_reward': None, 'subreddit_id': N...",[],False,96f6c79e-b853-11e5-a4cb-0ebdf030e05d,False,False,False,<NA>,[],False,<NA>,<NA>,<NA>,t5_2th52,<NA>,<NA>,#800080,mrbmiq,True,<NA>,AutoModerator,<NA>,1395,False,some_ads,False,[],False,<NA>,/r/wallstreetbets/comments/mrbmiq/unpinned_dai...,some_ads,False,https://www.reddit.com/r/wallstreetbets/commen...,9799713,1618480815.0,0,<NA>,False,<NA>,<NA>,<NA>,<NA>
28,<NA>,wallstreetbets,,t2_b80uuxt,False,<NA>,0,False,Ouchy. -50.25% Loss.,"[{'e': 'text', 't': 'Loss'}]",r/wallstreetbets,False,7,loss,0,26.0,<NA>,False,t3_mrkvkm,False,light,0.74,,public,42,3,{},140.0,<NA>,False,[],<NA>,True,False,<NA>,{},Loss,False,42,<NA>,True,https://b.thumbs.redditmedia.com/LZOHMVUvCjN2i...,False,,"[{'e': 'text', 't': 'Weaponized Autist'}]",{'gid_1': 1},image,<NA>,False,<NA>,1618540199.0,richtext,7,<NA>,<NA>,richtext,i.redd.it,False,<NA>,<NA>,confidence,<NA>,https://i.redd.it/xo7pr5furdt61.jpg,<NA>,False,False,False,False,False,{'images': [{'source': {'url': 'https://previe...,"[{'giver_coin_reward': None, 'subreddit_id': N...",[],False,7d5f3586-0a95-11e9-a050-0efea8d9040e,False,False,False,Weaponized Autist,[],False,<NA>,<NA>,<NA>,t5_2th52,<NA>,<NA>,#ff0000,mrkvkm,True,<NA>,SilentPhanto,<NA>,22,True,some_ads,False,[],False,dark,/r/wallstreetbets/comments/mrkvkm/ouchy_5025_l...,some_ads,False,https://i.redd.it/xo7pr5furdt61.jpg,9799716,1618511399.0,0,<NA>,False,<NA>,<NA>,<NA>,<NA>
12,<NA>,wallstreetbets,,t2_6juqb,False,<NA>,0,False,Big 4 Accounting Firm EY Has Announced an Alli...,"[{'e': 'text', 't': 'News'}]",r/wallstreetbets,False,7,news,0,<NA>,<NA>,False,t3_mrgyum,False,light,0.81,<NA>,public,95,1,{},<NA>,<NA>,False,[],<NA>,False,False,<NA>,{},News,False,95,<NA>,False,default,False,<NA>,[],{},<NA>,<NA>,False,<NA>,1618529065.0,richtext,7,<NA>,<NA>,text,stockhouse.com,False,<NA>,<NA>,confidence,<NA>,https:

In [15]:
final_df = df.loc[:,["title","created","num_comments","score","upvote_ratio","ups","downs"]]
final_df['created'] = pd.to_datetime(pd.to_numeric(df['created'])* 1e9).dt.date
final_df["num_comments"] = pd.to_numeric(df["num_comments"])
final_df["score"] = pd.to_numeric(df["score"])
final_df["upvote_ratio"] = pd.to_numeric(df["upvote_ratio"])
final_df["ups"] = pd.to_numeric(df["ups"])
final_df["downs"] = pd.to_numeric(df["downs"])
final_df

,title,created,num_comments,score,upvote_ratio,ups,downs
0,"Dave Portnoy-backed Buzz ETF adds GameStop, Pa...",2021-04-16,137,334,0.81,334,0
1,Citron did not learn their lesson,2021-04-16,73,295,0.93,295,0
2,$NIO unveil ET7 2.0 upcoming money glitch,2021-04-15,65,185,0.86,185,0
3,$NOK it off!,2021-04-15,71,157,0.87,157,0
4,There once was a time where SPY ruled the land...,2021-04-16,52,160,0.95,160,0
...,...,...,...,...,...,...,...
57,$JNJ at 165 my calls will print hard can we pi...,2021-04-15,12,1,0.52,1,0
58,Is Inovio back in the fight or not???,2021-04-16,11,0,0.46,0,0
59,Simple bear case for gold and gold miners. RGL...,2021-04-16,34,0,0.47,0,0
60,"Wow, is this a stonk with no naked short force...",2021-04-15,6,0,0.40,0,0


In [9]:
df.groupby('permalink').count()['subreddit'].unique()


array([1], dtype=int64)

In [10]:
%%time

data=[]
for url in df['permalink']:
    print(f"https://www.reddit.com{url}.json")
    res= r.get(f"https://www.reddit.com{url}.json",headers=user_agent)
    if res.status_code == 200:
        time.sleep(0.71)
        data.append((url,res.json()))
    else:
        print(res.json())
#         time.sleep(10)
        break

https://www.reddit.com/r/wallstreetbets/comments/mrm41b/dave_portnoybacked_buzz_etf_adds_gamestop/.json
https://www.reddit.com/r/wallstreetbets/comments/mrkmd2/citron_did_not_learn_their_lesson/.json
https://www.reddit.com/r/wallstreetbets/comments/mr9qf4/nio_unveil_et7_20_upcoming_money_glitch/.json
https://www.reddit.com/r/wallstreetbets/comments/mr6l2t/nok_it_off/.json
https://www.reddit.com/r/wallstreetbets/comments/mrqcgh/there_once_was_a_time_where_spy_ruled_the_lands/.json
https://www.reddit.com/r/wallstreetbets/comments/mrjjyo/hyln_pt_2_a_fair_warning/.json
https://www.reddit.com/r/wallstreetbets/comments/mrhzgr/rocket_companies_rkt_due_diligence/.json
https://www.reddit.com/r/wallstreetbets/comments/mrmecy/pltr_i_wont_give_up_whos_with_me/.json
https://www.reddit.com/r/wallstreetbets/comments/mrs413/perfectly_balanced_as_all_things_should_be/.json
https://www.reddit.com/r/wallstreetbets/comments/mror9z/monday_tsla_gains/.json
https://www.reddit.com/r/wallstreetbets/comments/mr

In [11]:
def get_text(d):
    if d.get("replies"):
        text=get_text(d['replies']['data'])
        return f"||{d.get('body','')}{text}"
    elif d.get("children"):
        temp=""
        for cell in d['children']:
            if type(cell) == str:
                text = cell
            elif cell.get("data"):
                text=get_text(cell['data'])
            else:
                text=""
            temp+=f"||{d.get('body','')}{text}"
        return temp
    
    elif d.get("body"):
        return f"||{d['body']}"
    
    else:
        return ""

In [12]:
final_data=[]

for obj in data:
    link=obj[0]
    thread=obj[1][1]
    comment_texts=[]
    main_comments = thread['data']['children']

    for c in main_comments:    
        if c.get('data'):
            text = get_text(c['data'])
            comment_texts.extend(text.split("||"))
    for text in comment_texts:
        if text:
            final_data.append((link,text))
print(len(final_data))

2195


In [13]:
final_data[0]

('/r/wallstreetbets/comments/mrm41b/dave_portnoybacked_buzz_etf_adds_gamestop/',
 "Isn't he the guy with the paper hands?")

In [14]:
comments_df=pd.DataFrame(final_data, columns=['link','text'])
comments_df.sample(20)

,link,text
445,/r/wallstreetbets/comments/mrmecy/pltr_i_wont_...,That's the spirit
507,/r/wallstreetbets/comments/mrmecy/pltr_i_wont_...,"Not ego, palantir is a great company. I like t..."
1270,/r/wallstreetbets/comments/mrg0kl/according_to...,"Yeah, I could have worded it a little less jer..."
543,/r/wallstreetbets/comments/mrmecy/pltr_i_wont_...,NEVER SELLING !! 💎👐
1067,/r/wallstreetbets/comments/mr5spq/coin_moon_po...,No shit.
40,/r/wallstreetbets/comments/mrm41b/dave_portnoy...,So long on $DICKS
1457,/r/wallstreetbets/comments/mro6xl/10k_tm/,Hoping they can wait till after june
1551,/r/wallstreetbets/comments/mrp16u/am_i_ape_now/,no? wrong ticker
1371,/r/wallstreetbets/comments/mrfyrq/rkt_operation/,Ever heard of a second picture? lol the inbred...
294,/r/wallstreetbets/comments/mr6l2t/nok_it_off/,Lol premium as a percentage of capital. Doesn’...
